In [3]:
import os
import sys

# CRITICAL: Add local source to path FIRST to use local changes
sys.path.insert(0, "/Users/ryanarman/code/oumi/src")
print(f"Using local source code from: /Users/ryanarman/code/oumi/src")

# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

Using local source code from: /Users/ryanarman/code/oumi/src
MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [4]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Path to the config file
config_path = "/Users/ryanarman/code/oumi/configs/examples/analyze/analyze.yaml"

# Path to your dataset file
dataset_path = "/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl"

# Load the config from YAML
config = AnalyzeConfig.from_yaml(
    config_path=config_path,
)

config.sample_count = 10

# Override the dataset settings to use your local file
config.dataset_path = dataset_path
config.dataset_name = None  # Clear dataset_name so it uses dataset_path instead

# Optionally update output path
config.output_path = "./analysis_output/banking77"

# IMPORTANT: Disable analyzers that require large model downloads or have issues
# IFD requires downloading Qwen model and may cause MPS crashes
# fasttext requires additional dependencies
# repr_diversity and question_diversity download embedding models
problematic_analyzers = ["repr_diversity", "question_diversity"]
# problematic_analyzers = ["ifd", "fasttext", "repr_diversity", "question_diversity"]
config.analyzers = [a for a in config.analyzers if a.id not in problematic_analyzers]
print(f"Running {len(config.analyzers)} analyzers: {[a.id for a in config.analyzers]}")

# Validate the configuration
config.finalize_and_validate()

# Create the analyzer
analyzer = DatasetAnalyzer(config)

Running 16 analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'fasttext', 'ifd', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-29 16:53:37,182][oumi][rank0][pid:86668][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2025-12-29 16:53:37,182][oumi.utils.analysis_utils][rank0][pid:86668][MainThread][INFO]][analysis_utils.py:225] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2025-12-29 16:53:37,183][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2025-12-29 16:53:37,304][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:304] Initialized sample analyzer: length
[2025-12-29 16:53:37,305][oumi][rank0][pid:86668][MainThread][IN

In [5]:
# Run the analysis
analyzer.analyze_dataset()

# The results are stored in analyzer object
if analyzer._analysis_results:
    print(
        f"Total conversations analyzed: {analyzer._analysis_results.conversations_analyzed}"
    )

[2025-12-29 16:53:37,320][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:347] Starting analysis of dataset: None
[2025-12-29 16:53:37,321][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:348] Using 16 sample analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'fasttext', 'ifd', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-29 16:53:37,322][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:371] Analyzing 10 of 8002 conversations
[2025-12-29 16:53:37,323][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:421] Converting conversation dataset with 8002 items
[2025-12-29 16:53:37,323][oumi][rank0][pid:86668][MainThread][INFO]][dataset_analyzer.py:428] Limiting analysis to first 10 items (dataset has 8002 total)


Converting Unknown Dataset to DataFrames: 100%|██████████| 10/10 [00:00<00:00, 1779.43item/s]


[2025-12-29 16:53:37,670][oumi][rank0][pid:86668][MainThread][INFO]][fasttext_analyzer.py:219] Initialized fast-langdetect for language detection
[2025-12-29 16:53:37,672][oumi][rank0][pid:86668][MainThread][INFO]][fasttext_analyzer.py:435] Analyzing language for column: conversation_text_content


[2025-12-29 16:53:37,878][INFO] - Downloading https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin to lid.176.bin (125.2M)
INFO:robust_downloader.downloader:Downloading https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin to lid.176.bin (125.2M)


  0%|          | 0.00/125M [00:00<?, ?B/s]

[2025-12-29 16:53:39,189][oumi][rank0][pid:86668][MainThread][INFO]][ifd_analyzer.py:153] Loading model for IFD analysis: Qwen/Qwen3-0.6B
[2025-12-29 16:53:40,250][oumi][rank0][pid:86668][MainThread][INFO]][ifd_analyzer.py:192] Loaded Qwen/Qwen3-0.6B on cpu (dtype: torch.float32)
[2025-12-29 16:53:40,251][oumi][rank0][pid:86668][MainThread][WARNING]][ifd_analyzer.py:629] Could not find instruction and response columns. For flat format, set instruction_column and response_column, or use columns like 'instruction'/'prompt' and 'response'/'output'. For conversation format, ensure 'text_content' and 'role' exist. Available columns: ['conversation_index', 'conversation_id', 'num_messages', 'conversation_text_content', 'conversation_text_content_length_token_count', 'conversation_text_content_diversity_unique_words_ratio', 'conversation_text_content_format_has_markdown', 'conversation_text_content_format_has_json', 'conversation_text_content_format_has_code_blocks', 'conversation_text_conten

/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


[2025-12-29 16:53:42,202][oumi][rank0][pid:86668][MainThread][WARNING]][dataset_analyzer.py:757] Failed to generate recommendations: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).
Total conversations analyzed: 10


In [6]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 166
len(merged_columns): 166


In [10]:
analyzer.analysis_df.head()

,conversation_index,conversation_id,num_messages,conversation_text_content,conversation_text_content_length_token_count,conversation_text_content_diversity_unique_words_ratio,conversation_text_content_format_has_markdown,conversation_text_content_format_has_json,conversation_text_content_format_has_code_blocks,conversation_text_content_format_code_block_count,...,conversation_structure_conversation_depth,conversation_structure_role_balance,conversation_structure_has_system_prompt,conversation_structure_avg_turn_length,conversation_structure_turn_length_variance,text_content_response_completeness_is_complete,text_content_response_completeness_score,text_content_response_completeness_ends_naturally,text_content_response_completeness_has_conclusion,text_content_response_completeness_truncation_type
0,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,None,NaN,None,None,None
1,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,None,NaN,None,None,None
2,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,False,0.5,False,False,
3,1,1,3,SYSTEM: You are a banking intent classifier. C...,1774,0.511602,True,False,False,0,...,1,0.5,True,6.0,25.00,None,NaN,None,None,None
4,1,1,3,SYSTEM: You are a banking intent classifier. C...,1774,0.511602,True,False,False,0,...,1,0.5,True,6.0,25.00,None,NaN,None,None,None


# Conv level

In [28]:
conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]
col = conv_columns[44]
print(f"col: {col}")
print(f"schema[col]: {schema[col]}")
print(f"row[col]: {row[col]}")

col: conversation_text_content_fasttext_language_name
schema[col]: {'type': <ColumnType.STRING: 'string'>, 'content_type': <ContentType.CATEGORICAL: 'categorical'>, 'description': 'Full language name'}
row[col]: Unknown


In [9]:
for i in range(len(conv_columns)):
    if "fasttext" in conv_columns[i]:
        print(i)

42
43
44
45
46
47


# Message level

In [17]:
msg_columns = analyzer.message_df.columns

for i in range(len(msg_columns)):
    if "fasttext" in msg_columns[i]:
        print(i)


44
45
46
47
48
49


In [35]:
row = analyzer.message_df.iloc[1]
col = msg_columns[44]
print(f"col: {col}")
print(f"schema[col]: {schema[col]}")
print(f"row[col]: {row[col]}")

col: text_content_fasttext_detected_language
schema[col]: {'type': <ColumnType.STRING: 'string'>, 'content_type': <ContentType.CATEGORICAL: 'categorical'>, 'description': 'ISO 639-1 language code detected'}
row[col]: unknown
